**TEST CODE REGRESSION**

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import warnings
from sklearn.svm import SVR 
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import pickle

warnings.filterwarnings('ignore') #ignore warning messages

In [2]:
# Get the file path from the user
file_path = input("Enter the file path: ")

# Check the file extension
if file_path.endswith('.csv'):
    # Read CSV file
    df = pd.read_csv(file_path)
elif file_path.endswith('.xls') or file_path.endswith('.xlsx'):
    # Read XLS/XLSX file
    df = pd.read_excel(file_path)
else:
    print("Invalid file format. Please provide a CSV or XLS/XLSX file.")
    exit()

Enter the file path: TestTrialDataset.xls


**Data analysis and preprocessing**

In [3]:
# Display the dataset read
df.head()

,ID,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002178,39.0,1,1,0,0,3,3,1,1,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
1,TRG002260,46.0,1,1,0,0,2,1,2,1,...,0.313170,0.313170,3.714752,0.007246,1192964.863,111.893409,0.002482,0.079986,0.005643,0.002184
2,TRG002268,47.0,0,0,0,1,3,3,1,0,...,0.429904,0.429904,3.330874,0.004290,6083001.390,170.357955,0.001556,0.045200,0.001885,0.001360
3,TRG002271,66.0,0,0,0,1,2,1,2,1,...,0.543363,0.543363,2.324037,0.004338,3224591.772,24.328720,0.010442,0.013187,0.000167,0.008646
4,TRG002272,43.0,0,0,0,1,2,1,1,1,...,0.262034,0.262034,3.261044,0.001756,4833932.159,294.198973,0.001111,0.143636,0.020441,0.001033


In [4]:
df.shape

(100, 118)

In [5]:
df.describe()

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
count,100.000000,100.00,100.000000,100.000000,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000
mean,51.041468,0.45,10.290000,10.350000,10.340000,2.35,1.580000,1.130000,0.480000,2.500000,...,0.395316,0.394949,2.635484,0.003003,2.245330e+07,329.040924,10000.040912,0.056549,0.006110,0.037659
std,10.632333,0.50,99.870754,99.864795,99.865791,0.50,0.806288,0.337998,0.502117,0.881917,...,0.171826,0.171321,0.748243,0.002318,7.232660e+07,2071.405376,99999.995868,0.051888,0.009264,0.127991
min,29.670089,0.00,0.000000,0.000000,0.000000,1.00,1.000000,1.000000,0.000000,1.000000,...,0.000021,0.000008,0.591673,0.000125,3.750289e+04,0.000000,0.000393,0.000000,0.000000,0.000000
25%,43.802875,0.00,0.000000,0.000000,0.000000,2.00,1.000000,1.000000,0.000000,2.000000,...,0.319481,0.319481,2.198418,0.001074,1.169844e+06,17.189726,0.001841,0.011883,0.000124,0.001585
50%,49.000000,0.00,0.000000,0.000000,0.000000,2.00,1.000000,1.000000,0.000000,2.000000,...,0.406160,0.406159,2.654426,0.002642,4.632876e+06,59.389960,0.004992,0.045385,0.002610,0.004358
75%,59.568788,1.00,1.000000,1.000000,1.000000,3.00,2.000000,1.000000,1.000000,3.000000,...,0.526561,0.526561,3.183633,0.004273,1.039883e+07,158.732133,0.015543,0.085816,0.008510,0.013397
max,79.603012,1.00,999.000000,999.000000,999.000000,3.00,3.000000,2.000000,1.000000,4.000000,...,0.857143,0.857143,4.947427,0.010431,5.488188e+08,20764.693790,1000000.000000,0.285100,0.060742,1.092132


In [6]:
df=df.replace(999, None)

In [7]:
#Taking the first 12 columns in order to handle the missing values efficiently. 
#The rest columns are values derived from the image scans
# the ID column is not needed.

Df_ = df.iloc[:,1:13]

In [8]:
for col in Df_.columns:
    Df_[col].fillna(Df_[col].mode()[0], inplace=True)

In [9]:
Df_imputed = pd.concat((Df_, df.iloc[:,13:]), axis=1)

In [11]:
Df_imputed

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,39.00000,1,1,0,0,3,3,1,1,2,...,0.444391,0.444391,3.032144,0.005612,2.372010e+06,59.459710,0.004383,0.032012,1.005722e-03,0.003685
1,46.00000,1,1,0,0,2,1,2,1,2,...,0.313170,0.313170,3.714752,0.007246,1.192965e+06,111.893409,0.002482,0.079986,5.642792e-03,0.002184
2,47.00000,0,0,0,1,3,3,1,0,2,...,0.429904,0.429904,3.330874,0.004290,6.083001e+06,170.357955,0.001556,0.045200,1.884625e-03,0.001360
3,66.00000,0,0,0,1,2,1,2,1,4,...,0.543363,0.543363,2.324037,0.004338,3.224592e+06,24.328720,0.010442,0.013187,1.670920e-04,0.008646
4,43.00000,0,0,0,1,2,1,1,1,2,...,0.262034,0.262034,3.261044,0.001756,4.833932e+06,294.198973,0.001111,0.143636,2.044052e-02,0.001033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,51.00000,0,0,0,1,2,1,1,1,1,...,0.527778,0.527778,1.500000,0.001450,1.419345e+06,0.993682,0.252276,0.002874,5.640000e-06,0.232840
96,62.12731,0,0,0,1,3,2,1,1,4,...,0.328724,0.328668,3.615818,0.000212,5.488188e+08,18.649891,0.013421,0.001173,9.660000e-07,0.013201
97,48.50000,1,1,1,0,3,3,1,0,2,...,0.397601,0.397601,3.401123,0.003057,6.264910e+06,205.220652,0.001349,0.066663,4.626845e-03,0.001174
98,38.50000,0,0,1,0,3,3,1,1,2,...,0.410970,0.410970,2.792586,0.005509,4.708709e+05,57.290026,0.005201,0.111502,1.218696e-02,0.004557


**Feature selection after using Random Forest for feature selection**

In [14]:
selected_column_names = ["Age", "original_firstorder_90Percentile",
       "original_firstorder_Kurtosis", "original_firstorder_Range",
       "original_firstorder_Variance",
       "original_glrlm_ShortRunHighGrayLevelEmphasis",
       "original_glszm_SizeZoneNonUniformity"]

In [15]:
feature_selected = Df_imputed[selected_column_names]

In [17]:
feature_selected.shape

(100, 7)

In [18]:
# Create a scaler object
scaler = MinMaxScaler()

# Scale the data
data_scaled = scaler.fit_transform(feature_selected)

In [19]:
model = pickle.load(open('svr_gridsearch_optimised_using_rf_for_fs.pkl', 'rb'))

In [20]:
#model prediction
y_pred = model.predict(data_scaled)

In [21]:
test_predictions = pd.DataFrame(y_pred, columns=['RelapseFreeSurvival (outcome)'])

In [22]:
to_excel_df = pd.DataFrame(df['ID'])

In [23]:
#Data after handling missing values
to_excel_df = pd.concat((to_excel_df, test_predictions),axis=1)
to_excel_df.to_excel('FinalTestRFS.xlsx',index = False, header=True)

**END**